# Benchmarks for the two-pole (2p) saturation formula
This notebook compares the compact 2p formulation to IAPWS-95 (liquid saturation) and Murphy–Koop 2005 (supercooled).

In [ ]:
from __future__ import annotations
import json
import math
from pathlib import Path
import sys

import numpy as np
import matplotlib.pyplot as plt


def find_project_root(start: Path) -> Path:
    for path in (start, *start.parents):
        if (path / 'pyproject.toml').exists():
            return path
    raise RuntimeError('Could not locate project root (pyproject.toml missing)')


if '__file__' in globals():
    start_path = Path(__file__).resolve().parent
else:
    start_path = Path.cwd()

PROJECT_ROOT = find_project_root(start_path)
sys.path.append(str(PROJECT_ROOT / 'src'))

from wsp2p import esat_water_hpa, coeffs

try:
    from iapws import IAPWS95
except Exception as exc:
    IAPWS95 = None
    print('iapws not available, liquid benchmarks will show NaN values:', exc)


In [ ]:

def iapws_saturation(T_c: np.ndarray) -> np.ndarray:
    if IAPWS95 is None:
        return np.full_like(T_c, np.nan, dtype=np.float64)
    T_k = T_c + 273.15
    out = np.full_like(T_c, np.nan, dtype=np.float64)
    mask = T_k >= 273.16
    if not np.any(mask):
        return out
    vals = []
    for temp in T_k[mask]:
        vals.append(IAPWS95(T=temp, x=0).P * 1e4)
    out[mask] = np.array(vals, dtype=np.float64)
    return out

def murphy_koop_water(T_c: np.ndarray) -> np.ndarray:
    T_k = T_c + 273.15
    ln_p = (
        54.842763
        - 6763.22 / T_k
        - 4.21 * np.log(T_k)
        + 0.000367 * T_k
        + np.tanh(0.0415 * (T_k - 218.8))
        * (53.878 - 1331.22 / T_k - 9.44523 * np.log(T_k) + 0.014025 * T_k)
    )
    return np.exp(ln_p) / 100.0  # Pa -> hPa

def rel_err(est: np.ndarray, ref: np.ndarray) -> np.ndarray:
    out = np.full_like(est, np.nan, dtype=np.float64)
    mask = np.isfinite(ref) & (ref > 0)
    out[mask] = (est[mask] - ref[mask]) / ref[mask] * 100.0
    return out

def metrics(label: str, ref: np.ndarray, est: np.ndarray) -> dict[str, float]:
    mask = np.isfinite(ref) & np.isfinite(est)
    if not np.any(mask):
        return {"label": label, "rmse_hpa": np.nan, "max_hpa": np.nan, "rmse_pct": np.nan, "max_pct": np.nan}
    diff = est[mask] - ref[mask]
    rmse_hpa = float(np.sqrt(np.mean(diff ** 2)))
    max_hpa = float(np.max(np.abs(diff)))
    mask_pos = ref[mask] > 0
    if np.any(mask_pos):
        pct = diff[mask_pos] / ref[mask][mask_pos] * 100.0
        rmse_pct = float(np.sqrt(np.mean(pct ** 2)))
        max_pct = float(np.max(np.abs(pct)))
    else:
        rmse_pct = np.nan
        max_pct = np.nan
    return {"label": label, "rmse_hpa": rmse_hpa, "max_hpa": max_hpa, "rmse_pct": rmse_pct, "max_pct": max_pct}

def reference_curve(T_c: np.ndarray) -> np.ndarray:
    ref = np.full_like(T_c, np.nan, dtype=np.float64)
    mask_warm = T_c >= 0.01
    if np.any(mask_warm):
        ref[mask_warm] = iapws_saturation(T_c[mask_warm])
    mask_cold = ~mask_warm
    if np.any(mask_cold):
        ref[mask_cold] = murphy_koop_water(T_c[mask_cold])
    return ref

def temperature_grid(t_min: float, t_max: float, step: float = 0.05) -> np.ndarray:
    if t_max < t_min:
        raise ValueError('t_max must be >= t_min')
    temps = np.arange(t_min, t_max + step, step, dtype=np.float64)
    if temps.size == 0:
        return temps
    if temps[-1] < t_max:
        temps = np.append(temps, t_max)
    elif temps[-1] > t_max:
        temps[-1] = t_max
    return temps

def metrics_over_range(label: str, t_min: float, t_max: float, step: float = 0.05) -> dict[str, float]:
    temps = temperature_grid(t_min, t_max, step)
    ref = reference_curve(temps)
    est = esat_water_hpa(temps)
    return metrics(label, ref, est)


In [ ]:

T_liq = np.arange(0.01, coeffs['domain_c']['max'] + 0.001, 0.05, dtype=np.float64)
T_sc = np.arange(coeffs['domain_c']['min'], 0.01, 0.05, dtype=np.float64)
es_liq = esat_water_hpa(T_liq)
es_sc = esat_water_hpa(T_sc)
ref_liq = iapws_saturation(T_liq)
ref_sc = murphy_koop_water(T_sc)
metrics_rows = [
    metrics_over_range('Supercooled vs Murphy-Koop (-25 to 0.01 °C)', -25.0, 0.01),
    metrics_over_range('Liquid vs IAPWS-95 (0.01 to 60 °C)', 0.01, 60.0),
    metrics('Supercooled vs Murphy-Koop (-40 to 0.01 °C)', ref_sc, es_sc),
    metrics('Liquid vs IAPWS-95 (0.01 to 100 °C)', ref_liq, es_liq),
]
metrics_rows


In [ ]:

FIG_DIR = PROJECT_ROOT / 'docs' / 'figures'
FIG_DIR.mkdir(parents=True, exist_ok=True)
T_all = np.linspace(coeffs['domain_c']['min'], coeffs['domain_c']['max'], 2000, dtype=np.float64)
es_all = esat_water_hpa(T_all)
ref_all = reference_curve(T_all)
plt.figure(figsize=(7, 4))
plt.plot(T_all, es_all, label='2p formula')
plt.plot(T_all, ref_all, label='Reference', linestyle='--')
plt.xlabel('Temperature (°C)')
plt.ylabel('Es (hPa)')
plt.title('Saturated vapor pressure over water')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(FIG_DIR / 'esat_curve.png', dpi=200)
plt.show()

abs_err = es_all - ref_all
plt.figure(figsize=(7, 4))
plt.plot(T_all, abs_err)
plt.xlabel('Temperature (°C)')
plt.ylabel('Error (hPa)')
plt.title('Absolute error vs references')
plt.axhline(0.0, color='k', linewidth=0.8)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / 'abs_error.png', dpi=200)
plt.show()

rel_pct = rel_err(es_all, ref_all)
plt.figure(figsize=(7, 4))
plt.plot(T_all, rel_pct)
plt.xlabel('Temperature (°C)')
plt.ylabel('Error (%)')
plt.title('Relative error vs references')
plt.axhline(0.0, color='k', linewidth=0.8)
plt.ylim(-0.05, 0.05)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / 'rel_error.png', dpi=200)
plt.show()

mask_zoom = (T_all >= -0.2) & (T_all <= 0.2)
plt.figure(figsize=(7, 4))
plt.plot(T_all[mask_zoom], abs_err[mask_zoom], label='abs err (hPa)')
plt.plot(T_all[mask_zoom], rel_pct[mask_zoom], label='rel err (%)')
plt.xlabel('Temperature (°C)')
plt.title('Zoom near 0 °C')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(FIG_DIR / 'zoom_triple_point.png', dpi=200)
plt.show()


In [ ]:
header = '| Benchmark | RMSE (hPa) | Max |err| (hPa) | RMSE (%) | Max |err| (%) |'
separator = '| --- | --- | --- | --- | --- |'
rows = [header, separator]
for row in metrics_rows:
    rows.append(f"| {row['label']} | {row['rmse_hpa']:.6f} | {row['max_hpa']:.6f} | {row['rmse_pct']:.6f} | {row['max_pct']:.6f} |")
markdown_table = '\n'.join(rows)
print(markdown_table)
with open(PROJECT_ROOT / 'docs' / 'figures' / 'benchmark_table.md', 'w', encoding='utf-8') as fh:
    fh.write(markdown_table + '\n')
markdown_table